In [1]:
# Importing Required Python Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [2]:
# Loading Reduced feature Training set
X_train_red = pd.read_csv('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

In [3]:
# Loading Reduced feature Test set
X_test_red = pd.read_csv('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

## Model_16: Weighted Aggregating Classifier 

####  Loading  the best Voting Classifier model & Neural Network (with Equal Nodes in all the Layers)

In [4]:
# Importing Joblib module
import joblib

In [5]:
import tensorflow
from tensorflow import keras

In [6]:
# Importing best Voting Classifier
voting_clf = joblib.load('Voting_Red.joblib')

In [7]:
# Importing best Neural Net Classifier
neural = keras.models.load_model('Best_model_Selu_eq_Learn.h5')

### Performing 10 fold class validations to get best weights for Voting and Neural Calssifiers 

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
# Initializing the 10 Fold object
cv_strat = StratifiedKFold(10,random_state=42)

In [10]:
# Defining the prediction matrices for the voting & Neural classifiers
y_preds_vote = np.empty((10, int(len(X_train_red)/10)), dtype=np.float32)
y_preds_neural = np.empty((10, int(len(X_train_red)/10)), dtype=np.float32)
y_true = np.empty((10, int(len(X_train_red)/10)), dtype=np.float32)

### Getting predictions of the Voting classifier for each fold of Training set.

In [11]:
# Computing  10 fold  CV predictions on the training set for the Voting Classifier
i = 0
for train_index, test_index in cv_strat.split(X_train_red, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train_red.loc[train_index], X_train_red.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    # Fitting the best Voting Classifier
    voting_clf.fit(X_tr, y_tr)
    y_pred = voting_clf.predict_proba(X_tst)[:,1]
    # Storing the fold predictions in a numpy array
    y_preds_vote[i,:] = y_pred
    # Storing the true fold values in a numpy array
    y_true[i,:] = y_tst.values.flatten()
    i = i+1  

In [14]:
#y_preds_vote.shape
y_true.shape


(10, 3295)

In [15]:
# Storing the copies of the Voting classifier Predictions & True Class labels for each instance of each test fold.
y_preds_vote_copy = y_preds_vote.copy()
y_true_copy = y_true.copy()

In [16]:
# Turning predictions & corresponding class labels into column vectors.
y_preds_vote = y_preds_vote.reshape(-1,1)
y_true = y_true.reshape(-1,1)

In [18]:
y_true_copy.shape

(10, 3295)

In [19]:
# Saving the Voting Predictions & True class labels for each test fold
pd.DataFrame(y_preds_vote_copy).to_csv('Voting_Preds_Folds.csv',index=False)
pd.DataFrame(y_true_copy).to_csv('True_Labels_Folds.csv',index=False)

### Getting predictions for the Neural classifier for each fold

In [20]:
# # Importing train test split from Sklearn to produce validation set
from sklearn.model_selection import train_test_split

In [21]:
# Defining the exponential decay learning rate.
def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /4 )

In [22]:
# Computing  10 fold  CV predictions on the training set for the Neural Network
i = 0
for train_index, test_index in cv_strat.split(X_train_red, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train_red.loc[train_index], X_train_red.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    
    # Splitting the Training set further into training & validation set.
    X_tr_r, X_val, y_tr_r, y_val = train_test_split(X_tr, y_tr, test_size=0.1, random_state=42, stratify=y_tr)
    
    # Loading fresh keras model on every iteration
    neural = keras.models.load_model('Best_model_Selu_eq_Learn.h5')
    
    # comiling & Training the Neural Net
    neural.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999), metrics=["accuracy"])
    
    # defining Checkpoints
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)# 2nd Callback
    lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
    
    # Fitting The model
    neural.fit(X_tr_r, y_tr_r, epochs=50, validation_data=(X_val,y_val),batch_size=32,
    class_weight={0: 1.0, 1: 10.0},callbacks=[checkpoint_cb,early_stopping_cb,lr_scheduler_cb])
    
    # Loading the best Neural model after training & before making predictions
    neural_best = keras.models.load_model('best_model.h5')    
    
    # Saving the predictions for every fold in a numpy array
    y_pred = neural_best.predict_proba(X_tst)
    y_preds_neural[i,:] = y_pred.flatten()
    i = i+1  

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 26689 samples, validate on 2966 samples
Epoch 1/50
26689/26689 [==============================] - 4s 140us/sample - loss: 1.1999 - accuracy: 0.6931 - val_loss: 1.1904 - val_accuracy: 0.7333
Epoch 2/50
26689/26689 [==============================] - 2s 78us/sample - loss: 1.1355 - accuracy: 0.7297 - val_loss: 1.0706 - val_accuracy: 0.8247
Epoch 3/50
26689/26689 [==============================] - 2s 78us/sample - loss: 1.1027 - accuracy: 0.7559 - val_loss: 1.0679 - val_accuracy: 0.7923
Epoch 4/50
26689/26689 [==============================] - 2s 77us/sample - loss: 1.0893 - accuracy: 0.7723 - val_loss: 1.0714 - val_accuracy: 0.7997
Epoch 5/50
26689/26689 [==============================] - 2s 77us/sample - loss: 1.0799 - accuracy: 0.7783 - val_loss: 1.1207 - val_accuracy: 0.5539
Epoch 6/50
26689/26689 [==============================] - 2s 78us/sample - loss: 1.0704 - accuracy: 0.7800 - val_loss: 1.0556 - val_accuracy: 0.7893
Epoch 

26689/26689 [==============================] - 2s 76us/sample - loss: 1.0395 - accuracy: 0.7867 - val_loss: 1.0826 - val_accuracy: 0.7913
Epoch 18/50
26689/26689 [==============================] - 2s 77us/sample - loss: 1.0395 - accuracy: 0.7867 - val_loss: 1.0826 - val_accuracy: 0.7913
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 26689 samples, validate on 2966 samples
Epoch 1/50
26689/26689 [==============================] - 3s 111us/sample - loss: 1.1792 - accuracy: 0.7074 - val_loss: 1.2914 - val_accuracy: 0.8493
Epoch 2/50
26689/26689 [==============================] - 2s 78us/sample - loss: 1.1244 - accuracy: 0.7376 - val_loss: 1.1638 - val_accuracy: 0.8436
Epoch 3/50
26689/26689 [==============================] - 2s 80us/sample - loss: 1.0875 - accuracy: 0.7655 - val_loss: 1.1066 - val_accuracy: 0.8173
Epoch 4/50
26689/26689 [==============================] - 2s 79us/sample - loss: 1.0730 - accuracy: 0.7848 - val_loss: 1.0720 - val_accuracy: 0.7980
Epoch 5/50
26689

In [23]:
# storing a copy of the Neural predictions for each fold in a numpy array & finally into a csv file
y_preds_neural_copy = y_preds_neural.copy()
pd.DataFrame(y_preds_neural_copy).to_csv('Neural_Preds_Folds.csv',index=False)

In [24]:
y_preds_neural_copy.shape

(10, 3295)

In [25]:
# Turning  neural predictions  into column vector.
y_preds_neural = y_preds_neural.reshape(-1,1)

In [26]:
# Confirming that voting classifier predictions , Neural predictions & True class labels have same shape
assert y_preds_neural.shape == y_preds_vote.shape == y_true.shape

In [27]:
# importing optuna
import optuna

In [28]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [29]:
# Declaring objective function for Weighted Aggregation Classifier
def objective_wrappper_auc(y_pred_cls_1, y_pred_cls_2, y_true):
    '''
    Optimizes weights for predictions from 2 classifiers to achieve maximum roc_auc score
      
    '''
    
    def objective(trial):
        w1 = trial.suggest_uniform('w1',-1,1)
        w2 = trial.suggest_uniform('w2',-1,1)
        
        y_final_pred = y_pred_cls_1 * w1 + y_pred_cls_2 * w2
        
        roc_auc_1 = roc_auc_score(y_true,y_final_pred)
        
                   
        return roc_auc_1
    
    return objective

In [30]:
# Defining the evaluation function for study's best parameters
def roc_auc_weights(y_pred_cls_1, y_pred_cls_2, y_true, obj_func, n_trials=200):
    ''' Computes the best hyper parameters of the classsifier on the Training set and returns 
    Optuna's study's best score & clasifier parameters'''
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(y_pred_cls_1, y_pred_cls_2, y_true),n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)


In [31]:
best_study_score,best_study_params = roc_auc_weights(y_preds_vote, y_preds_neural, y_true, objective_wrappper_auc,1000)

[I 2020-10-22 20:50:39,728] A new study created in memory with name: no-name-71c0ebaf-46fa-4c9e-8128-78407d00dc52
[I 2020-10-22 20:50:39,742] Trial 0 finished with value: 0.7712874965945355 and parameters: {'w1': -0.38504861498674847, 'w2': 0.7245313701325928}. Best is trial 0 with value: 0.7712874965945355.
[I 2020-10-22 20:50:39,753] Trial 1 finished with value: 0.1999509340407264 and parameters: {'w1': -0.7600424066570979, 'w2': -0.22465585974897562}. Best is trial 0 with value: 0.7712874965945355.
[I 2020-10-22 20:50:39,764] Trial 2 finished with value: 0.764979670962859 and parameters: {'w1': -0.40084637154122915, 'w2': 0.6740586199414385}. Best is trial 0 with value: 0.7712874965945355.
[I 2020-10-22 20:50:39,776] Trial 3 finished with value: 0.8002498326383827 and parameters: {'w1': 0.3536713490623571, 'w2': -0.021919771475948258}. Best is trial 3 with value: 0.8002498326383827.
[I 2020-10-22 20:50:39,785] Trial 4 finished with value: 0.7757564728515206 and parameters: {'w1': 0.

[I 2020-10-22 20:50:40,319] Trial 39 finished with value: 0.8003232675695422 and parameters: {'w1': 0.6134788446235964, 'w2': -0.02281434956403712}. Best is trial 21 with value: 0.8004135455438836.
[I 2020-10-22 20:50:40,335] Trial 40 finished with value: 0.7964980263417824 and parameters: {'w1': 0.13014865218094174, 'w2': 0.24400167968946015}. Best is trial 21 with value: 0.8004135455438836.
[I 2020-10-22 20:50:40,351] Trial 41 finished with value: 0.8003261699538979 and parameters: {'w1': 0.615519469406572, 'w2': -0.022120411632526878}. Best is trial 21 with value: 0.8004135455438836.
[I 2020-10-22 20:50:40,365] Trial 42 finished with value: 0.8001721086281197 and parameters: {'w1': 0.41299952659882944, 'w2': 0.09896369150957596}. Best is trial 21 with value: 0.8004135455438836.
[I 2020-10-22 20:50:40,380] Trial 43 finished with value: 0.8002435856015789 and parameters: {'w1': 0.8247166934765944, 'w2': -0.05266388463368245}. Best is trial 21 with value: 0.8004135455438836.
[I 2020-10

[I 2020-10-22 20:50:40,892] Trial 78 finished with value: 0.7994554546471763 and parameters: {'w1': 0.6844963278401524, 'w2': 0.3464271863722617}. Best is trial 47 with value: 0.8004242475103256.
[I 2020-10-22 20:50:40,908] Trial 79 finished with value: 0.7995341092632168 and parameters: {'w1': 0.5731177560996594, 'w2': 0.2744328811152519}. Best is trial 47 with value: 0.8004242475103256.
[I 2020-10-22 20:50:40,925] Trial 80 finished with value: 0.7993485547636991 and parameters: {'w1': 0.6523100536409039, 'w2': -0.12888059681249311}. Best is trial 47 with value: 0.8004242475103256.
[I 2020-10-22 20:50:40,939] Trial 81 finished with value: 0.8004212115241502 and parameters: {'w1': 0.4958673456604902, 'w2': 0.029848151657616666}. Best is trial 47 with value: 0.8004242475103256.
[I 2020-10-22 20:50:40,953] Trial 82 finished with value: 0.8003178589993302 and parameters: {'w1': 0.555254726154999, 'w2': 0.09274402735520043}. Best is trial 47 with value: 0.8004242475103256.
[I 2020-10-22 20

[I 2020-10-22 20:50:41,499] Trial 117 finished with value: 0.8003925562373363 and parameters: {'w1': 0.9525644519613478, 'w2': -0.002948036335391431}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:41,514] Trial 118 finished with value: 0.8003725297852818 and parameters: {'w1': 0.7742257438143308, 'w2': 0.10612144359343975}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:41,528] Trial 119 finished with value: 0.8001729609155892 and parameters: {'w1': 0.8147417208870933, 'w2': -0.06745942440300216}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:41,543] Trial 120 finished with value: 0.1998159132777137 and parameters: {'w1': -0.44159248688359276, 'w2': 0.03538513951275478}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:41,559] Trial 121 finished with value: 0.8002068589220805 and parameters: {'w1': 0.6978888768387492, 'w2': 0.15575378042947863}. Best is trial 84 with value: 0.8004263759255196.
[I 20

[I 2020-10-22 20:50:42,075] Trial 156 finished with value: 0.800421441872115 and parameters: {'w1': 0.8894042632482476, 'w2': 0.06145048162532479}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:42,089] Trial 157 finished with value: 0.8001289967030388 and parameters: {'w1': 0.8573069414521634, 'w2': -0.07898097913922114}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:42,103] Trial 158 finished with value: 0.8003997108451212 and parameters: {'w1': 0.985248776118595, 'w2': 0.10771073769861887}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:42,119] Trial 159 finished with value: 0.8004129005695824 and parameters: {'w1': 0.758672350530721, 'w2': 0.01142006242323703}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-22 20:50:42,134] Trial 160 finished with value: 0.8001746700974877 and parameters: {'w1': 0.6665318102210641, 'w2': 0.15876216836407564}. Best is trial 84 with value: 0.8004263759255196.
[I 2020-10-2

...

[I 2020-10-22 20:50:53,667] Trial 819 finished with value: 0.8003838951538621 and parameters: {'w1': 0.39025280414839464, 'w2': -0.0031771500458466014}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:53,687] Trial 820 finished with value: 0.8003538485653413 and parameters: {'w1': 0.9359590357502975, 'w2': 0.13813423134021607}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:53,706] Trial 821 finished with value: 0.7977756559351789 and parameters: {'w1': 0.8342131748344285, 'w2': 0.9610020324439121}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:53,726] Trial 822 finished with value: 0.2426086590048142 and parameters: {'w1': 0.03795783399352508, 'w2': -0.05818375728985343}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:53,747] Trial 823 finished with value: 0.8004059947375992 and parameters: {'w1': 0.8797995968273413, 'w2': 0.09046884354998164}. Best is trial 284 with value: 0.800426615487403.
[I 20

[I 2020-10-22 20:50:54,444] Trial 858 finished with value: 0.8003075624453062 and parameters: {'w1': 0.9644549334462054, 'w2': -0.041166245535252236}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:54,464] Trial 859 finished with value: 0.8004230450939495 and parameters: {'w1': 0.9068759524318228, 'w2': 0.0426197838085395}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:54,484] Trial 860 finished with value: 0.8003945280159146 and parameters: {'w1': 0.830048463853231, 'w2': -0.0014148949605126485}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:54,503] Trial 861 finished with value: 0.8004137528570519 and parameters: {'w1': 0.7701436764889812, 'w2': 0.0646557450830457}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:54,524] Trial 862 finished with value: 0.8003961220238306 and parameters: {'w1': 0.87375253691711, 'w2': 0.09985249194976656}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10

[I 2020-10-22 20:50:55,242] Trial 897 finished with value: 0.800204942427014 and parameters: {'w1': 0.8676317010528847, 'w2': 0.19487928006808616}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:55,261] Trial 898 finished with value: 0.8004258829808751 and parameters: {'w1': 0.9996596044955853, 'w2': 0.039389990280184514}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:55,281] Trial 899 finished with value: 0.8003313435691861 and parameters: {'w1': 0.9677452228696299, 'w2': 0.15422364525645474}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:55,300] Trial 900 finished with value: 0.8003919158699944 and parameters: {'w1': 0.9749308862249617, 'w2': 0.11574594800073836}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10-22 20:50:55,319] Trial 901 finished with value: 0.8004201150678381 and parameters: {'w1': 0.9985347563034538, 'w2': 0.06980289994480188}. Best is trial 284 with value: 0.800426615487403.
[I 2020-10

[I 2020-10-22 20:50:56,021] Trial 936 finished with value: 0.8004144807566205 and parameters: {'w1': 0.9072577884996794, 'w2': 0.07883074692989345}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,040] Trial 937 finished with value: 0.8004227871042291 and parameters: {'w1': 0.8583111807318956, 'w2': 0.04068705742411795}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,059] Trial 938 finished with value: 0.8001807098211232 and parameters: {'w1': 0.9554873951990849, 'w2': -0.0772224083005129}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,077] Trial 939 finished with value: 0.8003751465381613 and parameters: {'w1': 0.9968646247243587, 'w2': -0.013407499484993278}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,097] Trial 940 finished with value: 0.8004087635201356 and parameters: {'w1': 0.9992237709357853, 'w2': 0.09849895579599954}. Best is trial 902 with value: 0.8004266845917924.
[I 

[I 2020-10-22 20:50:56,800] Trial 975 finished with value: 0.8003921001483661 and parameters: {'w1': 0.7463557716690925, 'w2': -0.003548149977825616}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,819] Trial 976 finished with value: 0.800306309352378 and parameters: {'w1': 0.8009744397026577, 'w2': 0.13977032382998134}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,838] Trial 977 finished with value: 0.800141813263797 and parameters: {'w1': 0.9234294304832779, 'w2': -0.08240047769936143}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,856] Trial 978 finished with value: 0.8004216998618354 and parameters: {'w1': 0.8908473872355797, 'w2': 0.044820481324846284}. Best is trial 902 with value: 0.8004266845917924.
[I 2020-10-22 20:50:56,875] Trial 979 finished with value: 0.8003373372232286 and parameters: {'w1': 0.7725215715001911, 'w2': -0.025256720118393028}. Best is trial 902 with value: 0.8004266845917924.
[

In [32]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.8004266845917924


In [33]:
print(('The best study parameters for the classifier are: ',best_study_params))

('The best study parameters for the classifier are: ', {'w1': 0.9397785734580625, 'w2': 0.03493904313818527})


### Training the Tuned Weighted Aggregating Classifier on the whole Training set & making the predictions on the Test set.

In [34]:
# Training the voting classifier on whole Traing set
voting_clf.fit(X_train_red, y_train)

VotingClassifier(estimators=[('logistic_Reg',
                              LogisticRegression(C=0.12725888493400458,
                                                 class_weight={0: 1.0, 1: 9.0},
                                                 l1_ratio=0.9851193622801032,
                                                 n_jobs=5, penalty='elasticnet',
                                                 random_state=42,
                                                 solver='saga')),
                             ('Random_Forest',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=11,
                                                     min_samples_leaf=0.000264150675671259,
                                                     n_estimators=1560,
                                                     n_jobs=5,
                                                     rando...
                              

In [35]:
# Making Predictions on the Test Set using voting classifier
y_preds_vote_final = voting_clf.predict_proba(X_test_red)[:,1]

In [36]:
# Reloading best Neural Net Classifier 
neural = keras.models.load_model('Best_model_Selu_eq_Learn.h5')

In [37]:
# Making Predictions on the Test Set using Neural classifier
y_preds_neural_final = neural.predict_proba(X_test_red)

In [38]:
# Converting all test set  voting classifier  predictions into a column vector 
y_preds_vote_final = y_preds_vote_final.reshape(-1,1)

In [39]:
# combining the predictions from the Neural  & Voting classifier with the study's best votes.
y_combined_final = y_preds_vote_final * best_study_params['w1'] + y_preds_neural_final * best_study_params['w2']

In [40]:
# Computing the the final roc_auc score of the combined model 
print('The final test set roc_auc score of the combined voting & neural classifier using the best weights is: ',
     roc_auc_score(y_test,y_combined_final))

The final test set roc_auc score of the combined voting & neural classifier using the best weights is:  0.8173387453417614


### Calculating R_R ratio of the tuned Weighted Aggregating classifier using study's best weights.

In [41]:
# Asserting that all the Predictions & True Call label arrays have same shape
assert y_preds_vote_copy.shape == y_true_copy.shape == y_preds_neural_copy.shape

In [51]:
roc_auc_agg = []
for i in range(10):
    y_pred_combined = y_preds_vote_copy[i,:] * best_study_params['w1'] + y_preds_neural_copy[i,:] * best_study_params['w2']
    roc_auc_agg.append(roc_auc_score(y_true_copy[i,:],y_pred_combined))
    

In [52]:
print('The reward associated with the Aggregating classifier using roc_auc metric is: ',np.mean(roc_auc_agg))

The reward associated with the Aggregating classifier using roc_auc metric is:  0.8014073062606618


In [53]:
print('The risk associated with the Aggregating Classifier using roc_auc metric is: ',np.std(roc_auc_agg))

The risk associated with the Aggregating Classifier using roc_auc metric is:  0.016844751862311692


In [54]:
R_R_Ratio_Aggregator = np.mean(roc_auc_agg)/np.std(roc_auc_agg)

In [55]:
print('The reward risk ratio for the Aggregating Classifier using roc_auc metric is: ',R_R_Ratio_Aggregator)

The reward risk ratio for the Aggregating Classifier using roc_auc metric is:  47.57608261678961


## Observations: 
### 1) The test set roc_auc score for the tuned Weighted Aggregating classifier is nearly equal to that of best tuned Voting Classifier, owing to more weight being assigned to the voting classifier.
### 2) Similarly the R_R ratio of the Weighted Aggregating classifier is approx. equal (although more) to that of the best tuned Voting Classifier. So with added complexity the former doesn't offer any advantage over the latter.

### Best R_R Ratio for the tuned Weighted  Classifier using reduced feature set is:  47.57608261678961